In [12]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

In [13]:
# Read the Excel file
df = pd.read_excel("staff.xlsx")

# Read the project types
df_types = pd.read_excel("types.xlsx")

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Create dropdown menu options for project types
type_options = [{'label': project_type, 'value': project_type} for project_type in df_types['Type'].unique()]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [14]:
# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    dbc.Button("New Project", id="open", color="primary", className="mr-1"),
    dbc.Modal(
        [
            dbc.ModalHeader("New Project Details"),
            dbc.ModalBody(
                [
                    dcc.Input(id='type-input', type='text', placeholder='Type of Project'),
                    dcc.Input(id='duration-input', type='text', placeholder='Duration of Project')
                ]
            ),
            dbc.ModalFooter(
                [
                    dbc.Button("Close", id="close", className="ml-auto"),
                    dbc.Button("Save Changes", id="save-changes", className="ml-1")
                ]
            ),
        ],
        id="modal",
        size='lg',
        centered=True,
        backdrop='static'
    ),
    html.Div([
        html.Label('Type of Project:'),
        dcc.Dropdown(
            id='type-dropdown',
            options=type_options,
            value=type_options[0]['value'] if type_options else None  # Default value
        )
    ]),
    dcc.Dropdown(
        id='project-dropdown',
        options=project_options,
        value=project_options[0]['value']  # Default value
    ),
    dcc.Dropdown(
        id='job-dropdown',
        multi=True,
    ),
    dcc.Graph(id='deployment-plan'),
    html.Div(id='selected-project-type')  # Show the selected project type
])


In [15]:
# Define callback to update job dropdown options based on selected project
@app.callback(
    Output('job-dropdown', 'options'),
    [Input('project-dropdown', 'value')]
)
def update_job_dropdown(selected_project):
    # Filter the DataFrame based on the selected project
    filtered_data = df[df['Project'] == selected_project]

    # Get unique job titles for the selected project
    job_options = [{'label': 'Select All', 'value': 'all'}] + [{'label': job, 'value': job} for job in
                                                                filtered_data['Job'].unique()]

    return job_options


In [16]:
# Define callback to update the deployment plan based on selected project and job titles
@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('project-dropdown', 'value'),
     Input('job-dropdown', 'value')]
)
def update_plan(selected_project, selected_jobs):
    if not selected_project or not selected_jobs:
        # Return an empty figure if no project or job is selected
        return {'data': [], 'layout': {}}

    if 'all' in selected_jobs:
        # If 'Select All' is selected, set selected_jobs to all job titles for the selected project
        filtered_data = df[df['Project'] == selected_project]
        selected_jobs = filtered_data['Job'].unique()

    # Filter the DataFrame based on the selected project and selected job titles
    filtered_data = df[(df['Project'] == selected_project) & (df['Job'].isin(selected_jobs))]

    if filtered_data.empty:
        # Return an empty figure if the filtered data is empty
        return {'data': [], 'layout': {}}

    # Get unique job titles for the selected project
    unique_jobs = filtered_data['Job'].unique()

    # Get unique months for the selected project and sort them
    months = sorted(filtered_data['Month'].unique())

    # Create a matrix to store staff count for each job title in each month
    staff_matrix = []

    # Iterate through each job title and count staff for each month
    for job in unique_jobs:
        job_data = filtered_data[filtered_data['Job'] == job]
        staff_count = [
            job_data[job_data['Month'] == month].shape[0] if not job_data[job_data['Month'] == month].empty else None
            for month in months
        ]
        staff_matrix.append(staff_count)

    # Get the maximum staff count for the selected project
    max_value = max([item for sublist in staff_matrix for item in sublist if item is not None])

    # Create data for the heatmap
    data = [
        go.Heatmap(
            z=staff_matrix,
            x=months,
            y=unique_jobs,
            colorscale='Viridis',
            showscale=True,
            zmin=1,  # Set minimum value for the color scale
            zmax=max_value,  # Set maximum value for the color scale
            zmid=0,  # Set midpoint value for the color scale
            colorbar=dict(tickmode='array', tickvals=list(range(1, max_value + 1)), ticktext=list(range(1, max_value + 1)))
        )
    ]

    # Define layout for the heatmap
    layout = go.Layout(
        title=f"Deployment Plan for {selected_project}",
        xaxis=dict(title='Months'),
        yaxis=dict(title='Job Titles'),
    )

    return {'data': data, 'layout': layout}


In [17]:
# New callback for the modal popup
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks"), Input("save-changes", "n_clicks")],
    [dash.dependencies.State("modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open


In [18]:
# Callback to show the project type for the selected project
@app.callback(
    Output('selected-project-type', 'children'),
    [Input('project-dropdown', 'value')]
)
def display_project_type(selected_project):
    # Get the type of the selected project
    project_type = df_types[df_types['Project'] == selected_project]['Type'].values[0]
    return f'Type of Project: {project_type}'


In [19]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Add a callback for the "New Project" button
@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('open', 'n_clicks')],
    [dash.dependencies.State('type-dropdown', 'value')]
)
def generate_new_plan(n_clicks, selected_type):
    if n_clicks and selected_type:
        # Fetch historical data based on the selected type
        historical_data = generate_historical_data(selected_type)

        # Process historical data to generate a new deployment plan


        # Sample historical data generation function
        def generate_historical_data(selected_type):
            # Simulating fetching historical data based on selected_type
            # Replace this with your actual data retrieval logic

            # Generate dummy historical data
            np.random.seed(0)
            months = pd.date_range(start='2023-01-01', periods=12, freq='MS')
            job_titles = ['Job_A', 'Job_B', 'Job_C']
            historical_data = {
                'Month': np.repeat(months, len(job_titles)),
                'Job': np.tile(job_titles, len(months)),
                'Staff_Count': np.random.randint(1, 10, size=len(months) * len(job_titles))
            }
            historical_df = pd.DataFrame(historical_data)

            # Filter historical data based on selected_type
            filtered_historical_data = historical_df  # Filtering logic based on selected_type

            return filtered_historical_data

        # Here's a placeholder for a sample figure:
        new_figure = {
            'data': [go.Scatter(x=historical_data['Month'], y=historical_data['Staff_Count'], mode='lines')],
            'layout': {'title': f'New Deployment Plan for {selected_type}'}
        }

        return new_figure

    return {'data': [], 'layout': {}}  # Default empty figure


In [20]:
if __name__ == '__main__':
    app.run_server(debug=True)